In [168]:
from bs4 import BeautifulSoup
import requests

result  = requests.get("https://en.wikipedia.org/wiki/W16_engine")
content = result.content
soup = BeautifulSoup(content)

/home/irza/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/irza/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [169]:
#delete script tags
for x in soup.findAll('script'):
    x.extract()

In [170]:
#get all paragraph element

p_el = soup.find_all('p')

all_text=""

for x in p_el:    
    all_text=all_text+x.get_text()

all_text

"A W16 engine is a sixteen cylinder piston internal combustion engine in a four-bank W configuration. The most common layout for W16 engines consists of two 'offset double-row' banks of eight cylinders,[1] coupled to a single crankshaft. Other layouts, though, have been used before as well.Volkswagen Group is the only automotive manufacturer currently producing W16 engines. These engines are most notably used in the Bugatti Veyron and Bugatti Chiron.[2] French car maker Jimenez also used a custom 4.1L W16 made from four Yamaha motorcycle engines in the 1995 Jimenez Novia, a one-off French supercar.[3] The Volkswagen W16 engine was introduced with the mid-engined Bentley Hunaudieres concept car (Bentley Motors Limited has been a Volkswagen Group holding since 1998). This W16 was later used in the Audi Rosemeyer concept car, and in the aforementioned Bugatti Veyron and Chiron.Volkswagen Group's design is a stretched form of its W12 engine, which is itself based on technology from its VR6

In [171]:
#tokenize text
import nltk
import pandas as pd
import numpy as np

token = nltk.word_tokenize(all_text)
len(token)

491

In [172]:
prv = nltk.word_tokenize(all_text)
prv.pop()
prv.insert(0,'')
len(prv)

491

In [173]:
nxt = nltk.word_tokenize(all_text)
nxt.pop(0)
nxt.append('')
len(nxt)

491

In [174]:
#find all entities 
import re

entities=None

for x in p_el:
    a_el=x.find_all('a', {'href': re.compile(r'^/wiki/')})
    
    if len(a_el) > 0:
        if entities is None:
            entities=a_el
        else:            
            entities=entities+a_el

len(entities)

46

In [175]:
#get previous and next token of entity

ent_prev=[]
ent_next=[]
ent=[]

def get_token_from_sibling(el,src):    
    res=''
    
    if el is None:                
        if src=="prev":
            res=''
        elif src=="next":
            res=''
            
    elif isinstance(el, bs4.element.Tag):
        txt=el.get_text()
        tkn=nltk.word_tokenize(txt)

        if len(tkn) == 0:            
            if src=="prev":
                #print(t,src,type(el.previous_sibling))
                res=get_token_from_sibling(el.previous_sibling,src)
            elif src=="next":
                #print(t,src,type(el.next_sibling))
                res=get_token_from_sibling(el.next_sibling,src)
                
        elif src=="prev":
            res=tkn.pop()
        elif src=="next":
            res=tkn.pop(0)        
            
    elif isinstance(el, bs4.element.NavigableString):        
        tkn=nltk.word_tokenize(el)
        
        if len(tkn) == 0:               
            if src=="prev":
                #print(t,src,type(el.previous_sibling))
                res=get_token_from_sibling(el.previous_sibling,src)
            elif src=="next":
                #print(t,src,type(el.next_sibling))
                res=get_token_from_sibling(el.next_sibling,src)
                
        elif src=="prev":
            res=tkn.pop()
        elif src=="next":
            res=tkn.pop(0)
    
    return res

for x in entities:    
    ent_tkn=nltk.word_tokenize(x.get_text())
    
    for idx, t in enumerate(ent_tkn):    
        ent.append(t)
        
        if ((idx+1)==len(ent_tkn)) & (idx==0): 
            ent_prev.append(get_token_from_sibling(x.previous_sibling,"prev"))
            ent_next.append(get_token_from_sibling(x.next_sibling,"next"))
        
        elif idx==0: 
            ent_prev.append(get_token_from_sibling(x.previous_sibling,"prev"))
            ent_next.append(ent_tkn[idx+1])
                
        elif (idx+1)==len(ent_tkn): 
            ent_prev.append(ent_tkn[idx-1])
            ent_next.append(get_token_from_sibling(x.next_sibling,"next"))
        
        else:
            ent_prev.append(ent_tkn[idx-1])
            ent_next.append(ent_tkn[idx+1])
                            

In [176]:
print(len(ent_prev))
print(len(ent_next))
print(len(ent))
len(entities)

77
77
77


46

In [178]:
#save as text file

with open("Output.txt", "w") as text_file:    
    for idxt, t in enumerate(token):
        label="O"
        eq_ent=""
        eq_ent_prev=""
        eq_ent_next=""
        connector=""
        
        for idxet, et in enumerate(ent):
            if t == et:
                if (prv[idxt]==ent_prev[idxet]) & (nxt[idxt]==ent_next[idxet]):
                    label="I"
                    eq_ent=et
                    eq_ent_prev=ent_prev[idxet]
                    eq_ent_next=ent_next[idxet]
                    connector=' === '
        
        text_file.write(t + ' ' + label + connector + eq_ent_prev + " " + eq_ent + " " + eq_ent_next + " " '\n')